# Time Series Files For ESSP4

In [1]:
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.pyplot import errorbar

In [ ]:
# Base directory where DS1, DS2, ... live
essp_dir = "/work2/lbuc/data/ESSP4/ESSP4"

# Datasets to include
dataset_numbers = range(1, 10)   # DS1 ... DS9

# Output directories
outdir_dat = "/work2/lbuc/iara/GitHub/PyORBIT_examples/ESSP4/data"
fig_dir = "/work2/lbuc/iara/GitHub/ESSP/Figures"
os.makedirs(outdir_dat, exist_ok=True)
os.makedirs(fig_dir, exist_ok=True)

# Outlier settings
use_robust = False        # False = mean/std, True = median/MAD
sigma_threshold = 5        # Points beyond this (absolute) z-score are outliers

# Columns to test for outliers
cols_to_clip = [
    "RV [m/s]",
    "BIS [m/s]",
    "CCF FWHM [m/s]",
    "CCF Contrast",
    "H-alpha Emission",
    "CaII Emission"
]

# Fixed errors for BIS & FWHM when writing .dat
bis_err_val = 0.95
fwhm_err_val = 5.0

# Centering mode for plots: "instrument", "dataset", or "none"
center_mode = "instrument"   # choices: "instrument", "dataset", "none"

# Plot specification (per-dataset)
plot_specs = [
    ("RV [m/s]", "RV Err. [m/s]", "RV [m/s] (centered)"),
    ("CCF Contrast", 130, "CCF Contrast (centered)"),
    ("CCF FWHM [m/s]", 5.0, "FWHM [m/s] (centered)"),
    ("BIS [m/s]", 0.95, "BIS [m/s] (centered)"),
    ("H-alpha Emission", 0.001, "H-alpha (centered)"),
    ("CaII Emission", 0.003, "CaII (centered)"),
]

In [8]:
# Concat DS1 to DS9

df_list = []

for n in dataset_numbers:
    f = os.path.join(essp_dir, f"DS{n}", f"DS{n}_timeSeries.csv")
    if not os.path.exists(f):
        print(f"Missing: {f}")
        continue
    t = pd.read_csv(f)
    t["Dataset"] = f"DS{n}"
    
    # Convert FWHM km/s → m/s if present
    if "CCF FWHM [km/s]" in t.columns:
        t["CCF FWHM [m/s]"] = t["CCF FWHM [km/s]"] * 1000.0
        t.drop(columns=["CCF FWHM [km/s]"], inplace=True)
    if "CCF FWHM Err. [km/s]" in t.columns:
        t["CCF FWHM Err. [m/s]"] = t["CCF FWHM Err. [km/s]"] * 1000.0
        t.drop(columns=["CCF FWHM Err. [km/s]"], inplace=True)
    
    df_list.append(t)

if not df_list:
    raise RuntimeError("No datasets loaded.")
    
df_all = pd.concat(df_list, ignore_index=True)
print("Combined shape:", df_all.shape)
df_all.head()



Combined shape: (2669, 16)


,Standard File Name,Time [eMJD],RV [m/s],RV Err. [m/s],Exp. Time [s],Airmass,BERV [km/s],Instrument,CCF Contrast,CCF Contrast Err.,BIS [m/s],H-alpha Emission,CaII Emission,Dataset,CCF FWHM [m/s],CCF FWHM Err. [m/s]
0,DS1.001_spec_harpsn.fits,59337.993481,9.825036,0.103,300.000000,1.125065,-0.581131,harpsn,-457837.489595,543.369488,-51.516667,0.173834,0.100046,DS1,6856.193834,12.189541
1,DS1.002_spec_harpsn.fits,59337.997242,9.733036,0.103,300.000000,1.130287,-0.590573,harpsn,-457800.591871,550.192273,-50.216667,0.172751,0.100664,DS1,6857.638064,12.348111
2,DS1.003_spec_harpsn.fits,59338.004754,12.039036,0.103,300.000000,1.142726,-0.609293,harpsn,-457678.316202,557.232302,-49.500000,0.174994,0.107149,DS1,6856.078859,12.508341
3,DS1.004_spec_expres.fits,59338.301795,10.673016,0.110,185.271000,1.306683,-0.000004,expres,-354050.782988,544.182192,-76.200000,0.172216,0.103057,DS1,6692.954071,15.302120
4,DS1.005_spec_expres.fits,59338.304359,10.135016,0.110,185.887999,1.317263,-0.000004,expres,-354268.748737,553.370636,-73.850000,0.172323,0.100985,DS1,6690.431513,15.546367


## Outlier Detection

If use_robust = False:
\[
z = \frac{x - \mu}{\sigma}
\]

If use_robust = True:
\[
z = 0.6745 \frac{x - \text{median}}{\text{MAD}}
\]

Outlier if \[ |z| > \text{sigma\_threshold} ] in ANY listed column (per dataset).

In [9]:
def flag_outliers_per_group(group, columns, sigma=5, robust=False):
    # Start with all False
    outlier_flags = pd.Series(False, index=group.index)
    
    for col in columns:
        if col not in group.columns:
            continue
        x = group[col].astype(float)
        
        if robust:
            med = np.median(x)
            mad = np.median(np.abs(x - med))
            if mad == 0:
                continue
            z = 0.6745 * (x - med) / mad
        else:
            mean = x.mean()
            std = x.std(ddof=1)
            if std == 0 or np.isnan(std):
                continue
            z = (x - mean) / std
        
        outlier_flags |= (np.abs(z) > sigma)
    
    return outlier_flags


## Build Masks

In [10]:
mask_parts = []

for ds, g in df_all.groupby("Dataset"):
    m = flag_outliers_per_group(
        g,
        cols_to_clip,
        sigma=sigma_threshold,
        robust=use_robust
    )
    mask_parts.append(m)

outlier_mask = pd.concat(mask_parts).reindex(df_all.index)

df_outliers = df_all[outlier_mask].copy()
df_clean    = df_all[~outlier_mask].copy()

print(f"Total: {len(df_all)}  Inliers: {len(df_clean)}  Outliers: {len(df_outliers)}  ({len(df_outliers)/len(df_all)*100:.2f}%)")

# Optional per-dataset summary
summary = (
    df_all.assign(is_outlier=outlier_mask)
          .groupby("Dataset")["is_outlier"]
          .agg(total="count", outliers="sum")
)
summary["inliers"] = summary["total"] - summary["outliers"]
summary["outlier_pct"] = 100 * summary["outliers"] / summary["total"]
summary


Total: 2669  Inliers: 2659  Outliers: 10  (0.37%)


,total,outliers,inliers,outlier_pct
Dataset,,,,
DS1,288,1,287,0.347222
DS2,323,1,322,0.309598
DS3,323,1,322,0.309598
DS4,279,1,278,0.358423
DS5,280,0,280,0.000000
DS6,321,2,319,0.623053
DS7,290,1,289,0.344828
DS8,324,2,322,0.617284
DS9,241,1,240,0.414938


## Write PyORBIT .dat files

In [ ]:
for ds, sub in df_clean.groupby("Dataset"):
    instruments = sorted(sub["Instrument"].unique())
    inst_map = {inst: i for i, inst in enumerate(instruments)}
    
    time = sub["Time [eMJD]"].values
    offset_flag = sub["Instrument"].map(inst_map).astype(int).values
    jitter_flag = np.zeros(len(sub), dtype=int)
    subset_flag = -1 * np.ones(len(sub), dtype=int)
    
    # RV
    rv = sub["RV [m/s]"].values
    rv_err = sub["RV Err. [m/s]"].values
    rv_data = np.column_stack([time, rv, rv_err, jitter_flag, offset_flag, subset_flag])
    np.savetxt(os.path.join(outdir_dat, f"{ds}_RV.dat"),
               rv_data, fmt=["%.6f","%.6f","%.6f","%d","%d","%d"])
    
    # BIS
    bis = sub["BIS [m/s]"].values
    bis_err = np.full(len(sub), bis_err_val)
    bis_data = np.column_stack([time, bis, bis_err, jitter_flag, offset_flag, subset_flag])
    np.savetxt(os.path.join(outdir_dat, f"{ds}_BIS.dat"),
               bis_data, fmt=["%.6f","%.6f","%.6f","%d","%d","%d"])
    
    # FWHM
    fwhm = sub["CCF FWHM [m/s]"].values
    fwhm_err = np.full(len(sub), fwhm_err_val)
    fwhm_data = np.column_stack([time, fwhm, fwhm_err, jitter_flag, offset_flag, subset_flag])
    np.savetxt(os.path.join(outdir_dat, f"{ds}_FWHM.dat"),
               fwhm_data, fmt=["%.6f","%.6f","%.6f","%d","%d","%d"])
    
print("Finished writing .dat files.")

Finished writing .dat files.


## Per-Dataset Plots
Outliers = black points; Inliers colored by instrument. Values mean-centered (per inlier set).

In [13]:
plot_specs = [
    ("RV [m/s]", "RV Err. [m/s]", "RV [m/s] - mean"),
    ("CCF Contrast", 130, "CCF Contrast - mean"),
    ("CCF FWHM [m/s]", 5.0, "FWHM [m/s] - mean"),
    ("BIS [m/s]", 0.95, "BIS [m/s] - mean"),
    ("H-alpha Emission", 0.001, "H-alpha - mean"),
    ("CaII Emission", 0.003, "CaII - mean"),
]

for ds, sub_in in df_clean.groupby("Dataset"):
    sub_out = df_outliers[df_outliers["Dataset"] == ds]
    fig, axes = plt.subplots(len(plot_specs), 1, figsize=(12, 16), sharex=True)
    
    for ax, (col, yerr, ylabel) in zip(axes, plot_specs):
        if col not in sub_in.columns:
            continue
        mean_ref = sub_in[col].mean()
        
        # Inliers colored by instrument
        for inst, g in sub_in.groupby("Instrument"):
            if isinstance(yerr, str):
                yerr_vals = g[yerr]
            else:
                yerr_vals = yerr
            ax.errorbar(
                g["Time [eMJD]"],
                g[col] - mean_ref,
                yerr=yerr_vals,
                fmt='o',
                ms=4,
                label=inst,
                alpha=0.85
            )
        # Outliers in black
        if not sub_out.empty and col in sub_out.columns:
            if isinstance(yerr, str):
                out_yerr = sub_out[yerr]
            else:
                out_yerr = yerr
            ax.errorbar(
                sub_out["Time [eMJD]"],
                sub_out[col] - mean_ref,
                yerr=out_yerr,
                fmt='o',
                ms=4,
                color='black',
                alpha=0.6,
                label='Outlier'
            )
        ax.set_ylabel(ylabel)
    
    axes[-1].set_xlabel("Time [eMJD]")
    handles, labels = axes[0].get_legend_handles_labels()
    uniq = dict(zip(labels, handles))
    axes[0].legend(uniq.values(), uniq.keys(), fontsize=9, ncol=3)
    fig.suptitle(f"{ds} (black = outliers)", fontsize=15)
    plt.tight_layout(rect=[0,0,1,0.96])
    out_path = os.path.join(fig_dir, f"{ds}_activity.png")
    plt.savefig(out_path, dpi=150)
    plt.close(fig)
    print("Saved:", out_path)


Saved: /work2/lbuc/iara/GitHub/ESSP/Figures/DS1_activity.png
Saved: /work2/lbuc/iara/GitHub/ESSP/Figures/DS2_activity.png
Saved: /work2/lbuc/iara/GitHub/ESSP/Figures/DS3_activity.png
Saved: /work2/lbuc/iara/GitHub/ESSP/Figures/DS4_activity.png
Saved: /work2/lbuc/iara/GitHub/ESSP/Figures/DS5_activity.png
Saved: /work2/lbuc/iara/GitHub/ESSP/Figures/DS6_activity.png
Saved: /work2/lbuc/iara/GitHub/ESSP/Figures/DS7_activity.png
Saved: /work2/lbuc/iara/GitHub/ESSP/Figures/DS8_activity.png
Saved: /work2/lbuc/iara/GitHub/ESSP/Figures/DS9_activity.png


## Combined Plot (All Datasets)
Outliers in black; inliers colored by instrument; mean-centered.

In [14]:
combined_specs = [
    ("CCF Contrast", 130, "CCF Contrast - mean"),
    ("CCF FWHM [m/s]", 5.0, "FWHM [m/s] - mean"),
    ("BIS [m/s]", 0.95, "BIS [m/s] - mean"),
    ("H-alpha Emission", 0.001, "H-alpha - mean"),
    ("CaII Emission", 0.003, "CaII - mean"),
]

fig, axes = plt.subplots(len(combined_specs), 1, figsize=(12, 14), sharex=True)

for ax, (col, yerr, ylabel) in zip(axes, combined_specs):
    if col not in df_clean.columns:
        continue
    # Inliers per instrument
    for inst, g in df_clean.groupby("Instrument"):
        ax.errorbar(
            g["Time [eMJD]"],
            g[col] - g[col].mean(),
            yerr=yerr,
            fmt='.',
            alpha=0.8,
            label=inst
        )
    # Outliers
    if col in df_outliers.columns and not df_outliers.empty:
        global_mean = df_clean[col].mean()
        ax.errorbar(
            df_outliers["Time [eMJD]"],
            df_outliers[col] - global_mean,
            yerr=yerr,
            fmt='.',
            color='black',
            alpha=0.55,
            label='Outlier'
        )
    ax.set_ylabel(ylabel)

axes[-1].set_xlabel("Time [eMJD]")
handles, labels = axes[0].get_legend_handles_labels()
uniq = dict(zip(labels, handles))
axes[0].legend(uniq.values(), uniq.keys(), ncol=3, fontsize=9)
plt.tight_layout()
combined_path = os.path.join(fig_dir, "ALL_activity.png")
plt.savefig(combined_path, dpi=150)
plt.close()
print("Saved combined figure:", combined_path)


Saved combined figure: /work2/lbuc/iara/GitHub/ESSP/Figures/ALL_activity.png
